In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from matplotlib import pyplot as plt
import os

In [2]:
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms

In [3]:
torch.manual_seed(42)
np.random.seed(42)

In [4]:
models = {
    'Shallow MLP': nn.Sequential(
        nn.Linear(128*128, 64),
        nn.ReLU(),
        nn.Linear(64, 3)
    ),
    'Deep MLP': nn.Sequential(
        nn.Linear(128*128, 256),
        nn.ReLU(),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, 3)
    ),
    'Shallow CNN': nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Flatten(),
        nn.Linear(32*64*64, 3) 
    ),
    'Deep CNN': nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Flatten(),
        nn.Linear(64*32*32, 128),  
        nn.ReLU(),
        nn.Linear(128, 3)
    )

}

In [5]:
class XRay_CNN(Dataset):
    def __init__(self, path, dim):
        self.path = path
        self.normal = os.listdir(path + '/normal')
        self.pneumonia = os.listdir(path + '/pneumonia')
        self.tuberculosis = os.listdir(path + '/tuberculosis')
        self.t = transforms.Compose([transforms.ToTensor(),
                                     transforms.Grayscale(),
                                     transforms.Resize([dim, dim])])
    def __len__(self):
        return len(self.normal) + len(self.pneumonia) + len(self.tuberculosis)

    def __getitem__(self, idx):
        if idx < len(self.normal):
            img = plt.imread(self.path + '/normal/' + self.normal[idx])
            label = 0
        elif idx < len(self.normal) + len(self.pneumonia):
            img = plt.imread(self.path + '/pneumonia/' + self.pneumonia[idx - len(self.normal)])
            label = 1
        else:
            img = plt.imread(self.path + '/tuberculosis/' + self.tuberculosis[idx - len(self.normal) - len(self.pneumonia)])
            label = 2
        img = self.t(img)
        return img, label

In [6]:
class XRay_MLP(Dataset):
    def __init__(self, path, dim):
        self.path = path
        self.normal = os.listdir(path + '/normal')
        self.pneumonia = os.listdir(path + '/pneumonia')
        self.tuberculosis = os.listdir(path + '/tuberculosis')
        self.t = transforms.Compose([transforms.ToTensor(),
                                     transforms.Grayscale(),
                                     transforms.Resize([dim, dim]),
                                     transforms.Lambda(lambda x: x.view(-1))])
    def __len__(self):
        return len(self.normal) + len(self.pneumonia) + len(self.tuberculosis)

    def __getitem__(self, idx):
        if idx < len(self.normal):
            img = plt.imread(self.path + '/normal/' + self.normal[idx])
            label = 0
        elif idx < len(self.normal) + len(self.pneumonia):
            img = plt.imread(self.path + '/pneumonia/' + self.pneumonia[idx - len(self.normal)])
            label = 1
        else:
            img = plt.imread(self.path + '/tuberculosis/' + self.tuberculosis[idx - len(self.normal) - len(self.pneumonia)])
            label = 2
        img = self.t(img)
        return img, label

In [7]:
ds_train_CNN = XRay_CNN('dataset/train', 128)
ds_val_CNN = XRay_CNN('dataset/val', 128)

subset_size = 600
train_subset_CNN, _ = random_split(ds_train_CNN, [subset_size, len(ds_train_CNN) - subset_size])

subset_size = 300
val_subset_CNN, _ = random_split(ds_val_CNN, [subset_size, len(ds_val_CNN) - subset_size])


ds_train_MLP = XRay_MLP('dataset/train', 128)
ds_val_MLP = XRay_MLP('dataset/val', 128)

subset_size = 600
train_subset_MLP, _ = random_split(ds_train_MLP, [subset_size, len(ds_train_MLP) - subset_size])

subset_size = 300
val_subset_MLP, _ = random_split(ds_val_MLP, [subset_size, len(ds_val_MLP) - subset_size])

In [8]:
dl_train_CNN = DataLoader(train_subset_CNN, batch_size=32, shuffle=True)
dl_val_CNN = DataLoader(val_subset_CNN, batch_size=32, shuffle=True)

dl_train_MLP = DataLoader(train_subset_MLP, batch_size=32, shuffle=True)
dl_val_MLP = DataLoader(val_subset_MLP, batch_size=32, shuffle=True)

In [9]:
def train_model(model, train_dl, val_dl, epochs=1000, lr=0.1):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    train_loss_history = []
    train_acc_history = []
    val_loss_history = []
    val_acc_history = []

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0

        for batch_x, batch_y in train_dl:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            optimizer.zero_grad()
            output = model(batch_x)
            loss = criterion(output, batch_y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            train_correct += (predicted == batch_y).sum().item()

        train_loss_history.append(train_loss / len(train_dl))
        train_acc_history.append(train_correct / len(train_dl.dataset))

        model.eval()
        val_loss = 0.0
        val_correct = 0
        with torch.no_grad():
            for batch_x, batch_y in val_dl:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                output = model(batch_x)
                loss = criterion(output, batch_y)
                val_loss += loss.item()
                _, predicted = torch.max(output.data, 1)
                val_correct += (predicted == batch_y).sum().item()

        val_loss_history.append(val_loss / len(val_dl))
        val_acc_history.append(val_correct / len(val_dl.dataset))

        print(f"Epoch {epoch}, Train Loss: {train_loss_history[-1]:.4f}, Train Acc: {train_acc_history[-1]:.4f}, Val Loss: {val_loss_history[-1]:.4f}, Val Acc: {val_acc_history[-1]:.4f}")



    return train_loss_history, train_acc_history, val_loss_history, val_acc_history


In [10]:
train_loss_history, train_acc_history, val_loss_history, val_acc_history = train_model(models['Shallow MLP'], dl_train_MLP, dl_val_MLP, epochs=10, lr=0.001)

C:\Users\alumno\.conda\envs\py38ml\lib\site-packages\torchvision\transforms\functional.py:154: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:212.)
  img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()


Epoch 0, Train Loss: 1.7670, Train Acc: 0.4050, Val Loss: 1.0663, Val Acc: 0.6300
Epoch 1, Train Loss: 0.9193, Train Acc: 0.6250, Val Loss: 0.8010, Val Acc: 0.6300
Epoch 2, Train Loss: 0.7607, Train Acc: 0.6300, Val Loss: 0.7216, Val Acc: 0.6867
Epoch 3, Train Loss: 0.6688, Train Acc: 0.6900, Val Loss: 0.6539, Val Acc: 0.7000
Epoch 4, Train Loss: 0.6468, Train Acc: 0.7133, Val Loss: 0.6710, Val Acc: 0.7167
Epoch 5, Train Loss: 0.6309, Train Acc: 0.7267, Val Loss: 0.7094, Val Acc: 0.6633
Epoch 6, Train Loss: 0.6297, Train Acc: 0.7150, Val Loss: 0.6265, Val Acc: 0.7033
Epoch 7, Train Loss: 0.5969, Train Acc: 0.7283, Val Loss: 0.5990, Val Acc: 0.7500
Epoch 8, Train Loss: 0.5874, Train Acc: 0.7350, Val Loss: 0.5985, Val Acc: 0.7233
Epoch 9, Train Loss: 0.5631, Train Acc: 0.7467, Val Loss: 0.5776, Val Acc: 0.7500


In [11]:
train_loss_history, train_acc_history, val_loss_history, val_acc_history = train_model(models['Deep MLP'], dl_train_MLP, dl_val_MLP, epochs=10, lr=0.001)

Epoch 0, Train Loss: 0.9038, Train Acc: 0.5500, Val Loss: 0.8593, Val Acc: 0.6367
Epoch 1, Train Loss: 0.7401, Train Acc: 0.6683, Val Loss: 0.6355, Val Acc: 0.7367
Epoch 2, Train Loss: 0.6409, Train Acc: 0.7200, Val Loss: 0.6600, Val Acc: 0.7067
Epoch 3, Train Loss: 0.6397, Train Acc: 0.6950, Val Loss: 0.6707, Val Acc: 0.6433
Epoch 4, Train Loss: 0.6192, Train Acc: 0.7333, Val Loss: 0.6224, Val Acc: 0.7433
Epoch 5, Train Loss: 0.5684, Train Acc: 0.7517, Val Loss: 0.5703, Val Acc: 0.7300
Epoch 6, Train Loss: 0.5465, Train Acc: 0.7683, Val Loss: 0.5691, Val Acc: 0.7333
Epoch 7, Train Loss: 0.5416, Train Acc: 0.7633, Val Loss: 0.5863, Val Acc: 0.7200
Epoch 8, Train Loss: 0.5743, Train Acc: 0.7367, Val Loss: 0.5889, Val Acc: 0.7267
Epoch 9, Train Loss: 0.5397, Train Acc: 0.7700, Val Loss: 0.6276, Val Acc: 0.6700


In [12]:
train_loss_history, train_acc_history, val_loss_history, val_acc_history = train_model(models['Shallow CNN'], dl_train_CNN, dl_val_CNN, epochs=10, lr=0.001)

Epoch 0, Train Loss: 7.8537, Train Acc: 0.4517, Val Loss: 2.3542, Val Acc: 0.6200
Epoch 1, Train Loss: 2.4361, Train Acc: 0.6200, Val Loss: 2.0840, Val Acc: 0.6400
Epoch 2, Train Loss: 1.4594, Train Acc: 0.6650, Val Loss: 1.7049, Val Acc: 0.6833
Epoch 3, Train Loss: 0.9919, Train Acc: 0.6900, Val Loss: 1.0946, Val Acc: 0.6900
Epoch 4, Train Loss: 0.7797, Train Acc: 0.7317, Val Loss: 0.8727, Val Acc: 0.7133
Epoch 5, Train Loss: 0.6753, Train Acc: 0.7383, Val Loss: 0.6681, Val Acc: 0.7233
Epoch 6, Train Loss: 0.4845, Train Acc: 0.7867, Val Loss: 0.6046, Val Acc: 0.7067
Epoch 7, Train Loss: 0.5527, Train Acc: 0.7767, Val Loss: 0.8574, Val Acc: 0.6633
Epoch 8, Train Loss: 0.4224, Train Acc: 0.8200, Val Loss: 0.6943, Val Acc: 0.6900
Epoch 9, Train Loss: 0.4900, Train Acc: 0.7767, Val Loss: 0.8565, Val Acc: 0.6933


In [13]:
train_loss_history, train_acc_history, val_loss_history, val_acc_history = train_model(models['Deep CNN'], dl_train_CNN, dl_val_CNN, epochs=10, lr=0.001)

Epoch 0, Train Loss: 1.2712, Train Acc: 0.4883, Val Loss: 0.7743, Val Acc: 0.7233
Epoch 1, Train Loss: 0.7405, Train Acc: 0.6900, Val Loss: 0.7094, Val Acc: 0.6733
Epoch 2, Train Loss: 0.6511, Train Acc: 0.7233, Val Loss: 0.6607, Val Acc: 0.7167
Epoch 3, Train Loss: 0.6066, Train Acc: 0.7383, Val Loss: 0.5715, Val Acc: 0.7133
Epoch 4, Train Loss: 0.5764, Train Acc: 0.7333, Val Loss: 0.5859, Val Acc: 0.7367
Epoch 5, Train Loss: 0.5627, Train Acc: 0.7583, Val Loss: 0.6278, Val Acc: 0.7167
Epoch 6, Train Loss: 0.5505, Train Acc: 0.7583, Val Loss: 0.5872, Val Acc: 0.7067
Epoch 7, Train Loss: 0.4787, Train Acc: 0.7833, Val Loss: 0.5538, Val Acc: 0.7167
Epoch 8, Train Loss: 0.4482, Train Acc: 0.7750, Val Loss: 0.5621, Val Acc: 0.7367
Epoch 9, Train Loss: 0.4418, Train Acc: 0.7967, Val Loss: 0.5875, Val Acc: 0.7167


256x256 deep CNN

Epoch 0, Train Loss: 2.8587, Train Acc: 0.4200, Val Loss: 1.0547, Val Acc: 0.4500

Epoch 1, Train Loss: 0.8792, Train Acc: 0.6733, Val Loss: 0.7679, Val Acc: 0.6400

Epoch 2, Train Loss: 0.6183, Train Acc: 0.7433, Val Loss: 0.5706, Val Acc: 0.7400

Epoch 3, Train Loss: 0.4802, Train Acc: 0.8100, Val Loss: 0.5828, Val Acc: 0.7500

Epoch 4, Train Loss: 0.4373, Train Acc: 0.8200, Val Loss: 0.8224, Val Acc: 0.7400

Epoch 5, Train Loss: 0.3681, Train Acc: 0.8467, Val Loss: 0.7928, Val Acc: 0.7700

Epoch 6, Train Loss: 0.3037, Train Acc: 0.8633, Val Loss: 0.8480, Val Acc: 0.7200

Epoch 7, Train Loss: 0.2364, Train Acc: 0.9100, Val Loss: 0.6293, Val Acc: 0.7500

Epoch 8, Train Loss: 0.1749, Train Acc: 0.9233, Val Loss: 0.7078, Val Acc: 0.7000

Epoch 9, Train Loss: 0.1641, Train Acc: 0.9300, Val Loss: 0.7481, Val Acc: 0.7800